<a href="https://colab.research.google.com/github/lalavansh/Fake-News-Detection-Project-/blob/main/Fake_news_prediction4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Pre-processing

In [ ]:
# loading the dataset to a pandas Dataframe
news_dataset = pd.read_csv('/content/fake_and_real_news.csv.zip')

In [ ]:
news_dataset.shape

(9900, 2)

In [ ]:
# printing the first 5 rows of dataset
news_dataset.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
Text,0
label,0


In [ ]:
# replacing the null values with the empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# seperating the data & lable X = news_dataset.drop(columns='label', axis=1)Y = news_dataset['label']


In [ ]:
#print(X)
#print(Y)

                                                   Text
0      Top Trump Surrogate BRUTALLY Stabs Him In The...
1     U.S. conservative leader optimistic of common ...
2     Trump proposes U.S. tax overhaul, stirs concer...
3      Court Forces Ohio To Allow Millions Of Illega...
4     Democrats say Trump agrees to work on immigrat...
...                                                 ...
9895   Wikileaks Admits To Screwing Up IMMENSELY Wit...
9896  Trump consults Republican senators on Fed chie...
9897  Trump lawyers say judge lacks jurisdiction for...
9898   WATCH: Right-Wing Pastor Falsely Credits Trum...
9899   Sean Spicer HILARIOUSLY Branded As Chickensh*...

[9900 rows x 1 columns]
0       Fake
1       Real
2       Real
3       Fake
4       Real
        ... 
9895    Fake
9896    Real
9897    Real
9898    Fake
9899    Fake
Name: label, Length: 9900, dtype: object


Stemming:
Stemming is the process of reducing a word to its Root word.
example actor, actress, acting --> act  

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
!pip install swifter  # Install swifter if not available
import swifter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=a47ee094e3c2b75d6a2c495f13221a863a5c58aba635571e8dbaadb630d4dc7b
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


Pandas Apply:   0%|          | 0/9900 [00:00<?, ?it/s]

In [ ]:
news_dataset['Text'] = news_dataset['Text'].swifter.apply(stemming)

Pandas Apply:   0%|          | 0/9900 [00:00<?, ?it/s]

In [ ]:
 print(news_dataset['Text'])

0       top trump surrog brutal stab back pathet video...
1       u conserv leader optimist common ground health...
2       trump propo u tax overhaul stir concern defici...
3       court forc ohio allow million illeg purg voter...
4       democrat say trump agr work immigr bill wall d...
                              ...                        
9895    wikileak admit screw immen twitter poll hillar...
9896    trump consult republican senat fed chief candi...
9897    trump lawyer say judg lack jurisdict defam law...
9898    watch right wing pastor fal credit trump save ...
9899    sean spicer hilari brand chickensh bolt brief ...
Name: Text, Length: 9900, dtype: object


In [ ]:
#seperating the data and label
X = news_dataset['Text'].values
Y = news_dataset['label'].values

In [ ]:
print(X)
print(Y)

['top trump surrog brutal stab back pathet video look though republican presidenti candid donald trump lose support even within rank know thing get bad even top surrog start turn exactli happen fox news newt gingrich call trump pathet gingrich know trump need keep focu hillari clinton even remot want chanc defeat howev trump hurt feel mani republican support sexual assault women turn includ hou speaker paul ryan r wi made trump lash parti gingrich said fox news look first let say trump admir tri help much big trump littl trump littl trump frankli pathet mean mad get phone call trump refer fact paul ryan call congratul debat probabl win despit trump ego tell gingrich also ad donald trump one oppon name hillari clinton name paul ryan anybodi el trump seem realiz person mad truli worst enemi ultim lead defeat one blame watch via politico featur photo joe raedl getti imag'
 'u conserv leader optimist common ground healthcar washington reuter republican u hou repr could achiev common ground

In [ ]:
Y.shape

(9900,)

In [ ]:
#converting texual data to numerical data
np.vectorize = TfidfVectorizer()
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 282)	0.04782189145217878
  (0, 358)	0.09689879827014561
  (0, 961)	0.031464347982671746
  (0, 1433)	0.08352469440225876
  (0, 1836)	0.07292077385163638
  (0, 2235)	0.03955855092653222
  (0, 2275)	0.059710831347251046
  (0, 3185)	0.05642764229452826
  (0, 3402)	0.06511886234050777
  (0, 4329)	0.08398036418367025
  (0, 4791)	0.1029719170345693
  (0, 4872)	0.056267329546417194
  (0, 5426)	0.06876899106553187
  (0, 6103)	0.09893505150837939
  (0, 6626)	0.09822910916613485
  (0, 7962)	0.06549203134761152
  (0, 8075)	0.1533163415881811
  (0, 8381)	0.0570005109051573
  (0, 9059)	0.04562938877562089
  (0, 9900)	0.08861815708431664
  (0, 9989)	0.06565441267641081
  (0, 10285)	0.08243455810429008
  (0, 10709)	0.10587480893846517
  (0, 10786)	0.06623483103003042
  (0, 11008)	0.04534621752614209
  :	:
  (9899, 37536)	0.03575994718200495
  (9899, 37629)	0.030665967021491217
  (9899, 37755)	0.01844854562163104
  (9899, 37756)	0.026432965770461836
  (9899, 37762)	0.029608792648609716
  (9899, 3

Spliting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()


In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score on training data : ', training_data_accuracy)

Accuracy score on training data :  0.9959595959595959


In [ ]:
# accuracy score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score on test data : ', test_data_accuracy)

Accuracy score on test data :  0.9868686868686869


Making a predictive System

In [ ]:
X_new = X_test[13]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
print(Y_test[13])

1


In [ ]:
!git clone https://github.com/your-username/your-repo.git
!cp your_notebook.ipynb your-repo/
!cd your-repo && git add . && git commit -m "Updated project" && git push


Cloning into 'your-repo'...
fatal: could not read Username for 'https://github.com': No such device or address
cp: cannot stat 'your_notebook.ipynb': No such file or directory
/bin/bash: line 1: cd: your-repo: No such file or directory
